In [35]:
import json

def extract_headlines_and_sarcasm(json_file_path):
    with open(json_file_path, 'r') as file:
        data = file.readlines()

    articles_list = []
    for entry in data:
        article_data = json.loads(entry)
        article = [
            article_data["headline"],
            article_data["is_sarcastic"]
        ]
        articles_list.append(article)

    return articles_list

json_file_path = 'Sarcasm_Headlines_Dataset.json'  # Replace with the actual JSON file path
articles = extract_headlines_and_sarcasm(json_file_path)

print(bool(articles))


True


In [7]:
print(len(articles))

26709


In [8]:
import spacy
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the English model from spaCy
nlp = spacy.load("en_core_web_trf")

# Sample dataset
data = articles  # Sarcastic

# Separate text and labels
texts, labels = zip(*data)

# Preprocess and tokenize text using spaCy
tokenized_texts = [" ".join([token.lemma_ for token in nlp(text)]) for text in texts]

# Convert labels to numerical values
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tokenized_texts, encoded_labels, test_size=0.2, random_state=42)

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

max_sequence_length = max(max(map(len, X_train_sequences)), max(map(len, X_test_sequences)))
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=max_sequence_length)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_padded, y_train))
train_dataset = train_dataset.batch(32)

# Train the model
model.fit(train_dataset, epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")


Epoch 1/10
668/668 [==============================] - 10s 12ms/step - loss: 0.4073 - accuracy: 0.8084
Epoch 2/10
668/668 [==============================] - 5s 7ms/step - loss: 0.2048 - accuracy: 0.9208
Epoch 3/10
668/668 [==============================] - 5s 7ms/step - loss: 0.1060 - accuracy: 0.9633
Epoch 4/10
668/668 [==============================] - 5s 7ms/step - loss: 0.0555 - accuracy: 0.9812
Epoch 5/10
668/668 [==============================] - 5s 7ms/step - loss: 0.0394 - accuracy: 0.9862
Epoch 6/10
668/668 [==============================] - 5s 7ms/step - loss: 0.0265 - accuracy: 0.9910
Epoch 7/10
668/668 [==============================] - 5s 7ms/step - loss: 0.0176 - accuracy: 0.9937
Epoch 8/10
668/668 [==============================] - 5s 8ms/step - loss: 0.0164 - accuracy: 0.9946
Epoch 9/10
668/668 [==============================] - 5s 8ms/step - loss: 0.0135 - accuracy: 0.9956
Epoch 10/10
167/167 [==============================] - 1s 4ms/step - loss: 1.0488 - accuracy: 0.82

In [29]:
model.save("sarcastic_model")

INFO:tensorflow:Assets written to: sarcastic_model\assets


INFO:tensorflow:Assets written to: sarcastic_model\assets


In [30]:
tf.keras.models.load_model("sarcastic_model")

In [41]:
new_text = "Cow does not give milk human does - Tarek"
new_text_tokens = [" ".join([token.lemma_ for token in nlp(new_text)])]
new_text_sequences = tokenizer.texts_to_sequences(new_text_tokens)
new_text_padded = tf.keras.preprocessing.sequence.pad_sequences(new_text_sequences, maxlen=max_sequence_length)
predictions = model.predict(new_text_padded)
predicted_label = "sarcastic" if predictions > 0.5 else "non-sarcastic"
print(predictions,predicted_label)

1/1 [==============================] - 0s 18ms/step
[[0.9999771]] sarcastic
